In [ ]:
!pip install -q langchain==0.0.147
!pip install -q openai
!pip install -q pypdf
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q aifactory
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install googletrans==4.0.0-rc1

In [ ]:
import os
from glob import glob
import shutil
import requests
import json
import pickle

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PDFMinerLoader
from langchain.prompts import PromptTemplate, BaseChatPromptTemplate, StringPromptTemplate

from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain, ConversationalRetrievalChain, ChatVectorDBChain
from langchain.agents import AgentExecutor, AgentType, Tool, ZeroShotAgent, initialize_agent, LLMSingleActionAgent, AgentOutputParser
from typing import List, Union, Any, Optional
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain import LLMChain
import re
import queue
import threading
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.tracers import LangChainTracer
from langchain.memory import ConversationBufferWindowMemory


In [ ]:
os.environ["OPENAI_API_KEY"] = "api-key"

TASK_ID = 2449
TASK_KEY = "35711198-94bd-4321-af54-e907c651d698"

# Callback

In [ ]:
# Streaming
class ChainStreamHandler(StreamingStdOutCallbackHandler):
    def __init__(self, gen):
        super().__init__()
        self.gen = gen

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        self.gen.send(token)


class ThreadedGenerator:
    def __init__(self):
        self.queue = queue.Queue()

    def __iter__(self):
        return self

    def __next__(self):
        item = self.queue.get()
        if item is StopIteration:
            raise item
        return item

    def send(self, data):
        self.queue.put(data)

    def close(self):
        self.queue.put(StopIteration)



In [ ]:
from googletrans import Translator
def google_translate(text : str = None) -> str:
    print(text)
    translator = Translator()
    result = translator.translate(text, dest="ko")
    return result.text

# Tools

def Node_check(text):
    node_dict = {
        "서울" : [
            {"node_id" : 1,
                "run" : "ON"},

                {"node_id" : 2,
                "run" : "OFF"},

                {"node_id" : 3,
                "run" : "OFF"}
        ],

        "인천" : [
            {"node_id" : 1,
                "run" : "ON"},

                {"node_id" : 2,
                "run" : "OFF"},
        ],

        "대구" : [
            {"node_id" : 1,
                "run" : "ON"},
            

        ],   
    }

    return node_dict


def List_check(text):
    return ['Node System', 'List System', 'State System']

def State_check(text):
    State_dict = {
        "서울" : [
            {"state_id" : 1,
             "state" : 'NORMAL'},

             {"state_id" : 2,
              "state" : 'ABNORMAL'},

              {"state_id" : 3,
               "state" : 'NORMAL'}
        ],

        "인천" : [
            {"state_id" : 1,
             "state" : 'NORMAL'},

             {"state_id" : 2,
              "state" : 'ABNORMAL'},
        ],

        "대구" : [
            {"state_id" : 1,
             "state" : 'NORMAL'},


        ],   
    }
    return State_dict





def get_tools(g):
    if g == -1:
        manager = None
    else:
        manager = CallbackManager([ChainStreamHandler(g)])


    tools = []
    
    Node_tool = Tool(
        name="Node System",
        func=Node_check,
        description="시스템 노드에 대한 답변을 할때 사용합니다. ",
        callback_manager=manager,
        return_direct=False,
    )
    
    List_tool = Tool(
        name="List System",
        func=List_check,
        description="사용가능한 도구 목록에 대한 답변을 할때 사용합니다.",      
        callback_manager=manager,
        return_direct=False,
    )
    
    State_tool = Tool(
        name="State System",
        func=State_check,
        description="시스템 상태 목록에 대한 답변을 할때 사용합니다.",      
        callback_manager=manager,
        return_direct=False,
    )


    tools.append(Node_tool)
    tools.append(List_tool)
    tools.append(State_tool)

    return tools


In [ ]:

def get_agent(g):
    tools = get_tools(g)

    PREFIX = """\n\nHuman: You are KTChat, a large language model using Tools.
You have access to the following tools.  Only one tool can be used for user questions.
You MUST answer in Korean and in Markdown format:"""

    SUFFIX = '''CHAT HISTORY: """
{chat_history}
"""
Question: """
{input}
"""
Thought: """
{agent_scratchpad}
"""
'''
    
    # manager = None
    if g == -1:
        manager = None
    else:
        manager = CallbackManager([ChainStreamHandler(g)])


    streaming_llm = ChatOpenAI(
        streaming=True,
        verbose=True,
        model_name="gpt-3.5-turbo",
        callback_manager=manager,
        temperature=0.7,
        # max_tokens=2048
        )

    agent_kwargs={
        "input_variables": ["input", "agent_scratchpad", "chat_history"],
        "prefix": PREFIX,
        "suffix": SUFFIX,
        }

    agent = initialize_agent(tools, streaming_llm,
                             agent="zero-shot-react-description",
                             max_iterations=3,
                             callback_manager=manager,
                             agent_kwargs=agent_kwargs,
                             early_stopping_method="generate"
                             #  memory=memory,
                             )

    return agent


In [ ]:
def run_chain(g, data):
    try:
        ag = get_agent(g)
        ag(data)["output"]
    finally:
        g.close()

def chat(data):
    g = ThreadedGenerator()
    threading.Thread(target=run_chain, args=(g, data)).start()
    return g

def submit():
    return chat

In [ ]:
import aifactory.score as aif
if __name__ == "__main__":
    if os.path.isfile("aif.zip"):
        os.remove("aif.zip")
        
    aif.submit(model_name="KT_chat",
               key=TASK_KEY,
               func=submit
               )

file : task-jvsc-dc682e52-3b3b-43c2-8b63-1d327033aa5829ae43f9-d862-4eaf-85e0-2415d9e2b98e
jupyter notebook
request id : 447 processing...
score = 1g... 
